# **TD5**

# **Partie 2: RNN with Keras**

source: https://towardsdatascience.com/how-our-device-thinks-e1f5ab15071e

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#sentence = open("/content/drive/My Drive/DataSet/texte/LordOfTheRingPrologue.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap1.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap2.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap3.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap4.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap5.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap6.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap7.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap8.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap9.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap10.txt", encoding="latin-1").read()
#sentence = sentence + open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap11.txt", encoding="latin-1").read()
sentence = open("/content/drive/My Drive/DataSet/texte/LordOfTheRingChap12.txt", encoding="latin-1").read()

In [ ]:
sentence.replace('. ', ' \n')

'When Frodo came to himself he was still clutching the Ring desperately \nHe was lying by the fire, which was now piled high and burning brightly \nHis three companions were bending over him \n\'What has happened? Where is the pale king?\' he asked wildly \nThey were too overjoyed to hear him speak to answer for a while; nor did they understand his question \nAt length he gathered from Sam that they had seen nothing but the vague shadowy shapes coming towards them \nSuddenly to his horror Sam found that his master had vanished; and at that moment a black shadow rushed past him, and he fell \nHe heard Frodo\'s voice, but it seemed to come from a great distance, or from under the earth, crying out strange words \nThey saw nothing more, until they stumbled over the body of Frodo, lying as if dead, face downwards on the grass with his sword beneath him \nStrider ordered them to pick him up and lay him near the fire, and then he disappeared \nThat was now a good while ago.\n     Sam plainly

In [ ]:
tokenizer = Tokenizer() #instantiating the tokenizer
corpus = sentence.lower().split("\n") #converting the sentence to lowercase
tokenizer.fit_on_texts(corpus) #creates tokens for each words 
total_words = len(tokenizer.word_index) + 1 #calculating total number of words in the initial sentence

In [ ]:
input_sequences = [] #training features (x) will be a list

for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0] #converts each sentence as its tokenized equivalent
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1] #generating n gram sequences
		input_sequences.append(n_gram_sequence) #appending each n gram sequence to the list of our features (xs)


In [ ]:
max_sequence_len = max([len(x) for x in input_sequences]) #calculating the length of the longest sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) #pre-pading each value of the input_sequence
xs, labels = input_sequences[:,:-1],input_sequences[:,-1] #creating xs and their labels using numpy slicing
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words) #creating one hot encoding values


In [ ]:
model = Sequential() #creating a sequential model
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1)) #adding an embedding layer with 64 as the embedding dimension
model.add(Bidirectional(LSTM(20))) #adding 20 LSTM units
model.add(Dense(total_words, activation='softmax')) #creating a dense layer with 54 output units (total_words) with softmax activation

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #compiling the model with adam optimiser
history = model.fit(xs, ys, epochs=800, verbose=1) #training for 500 epochs

Epoch 1/800
272/272 [==============================] - 6s 21ms/step - loss: 6.4596 - accuracy: 0.0628
Epoch 2/800
272/272 [==============================] - 5s 20ms/step - loss: 5.9751 - accuracy: 0.0635
Epoch 3/800
272/272 [==============================] - 5s 20ms/step - loss: 5.8857 - accuracy: 0.0638
Epoch 4/800
272/272 [==============================] - 5s 20ms/step - loss: 5.8283 - accuracy: 0.0647
Epoch 5/800
272/272 [==============================] - 6s 20ms/step - loss: 5.7867 - accuracy: 0.0680
Epoch 6/800
272/272 [==============================] - 5s 20ms/step - loss: 5.7500 - accuracy: 0.0678
Epoch 7/800
272/272 [==============================] - 6s 20ms/step - loss: 5.7104 - accuracy: 0.0704
Epoch 8/800
272/272 [==============================] - 6s 20ms/step - loss: 5.6663 - accuracy: 0.0782
Epoch 9/800
272/272 [==============================] - 6s 21ms/step - loss: 5.6221 - accuracy: 0.0851
Epoch 10/800
272/272 [==============================] - 6s 20ms/step - loss: 5.572

In [ ]:
#Pour sauvegarder le modèle
model.save('/content/drive/My Drive/IA_model/textgeneration_LOTR_NLP_Projet2')
#pour utiliser un modèle sauvegardé
#model = tf.keras.models.load_model('/content/drive/My Drive/IA_model/textgeneration_LOTR_NLP_Projet')

In [ ]:
# plot diagnostic learning curves
import sys
from matplotlib import pyplot
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    #pyplot.plot(history.history['val_loss'], color='orange', label='test')
 
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Fbeta')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    #pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 
    # show the figure


    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig("/content/drive/My Drive/DataSet/file_plot.png", format = "png")
    #plt.savefig("test.png", format="png")
    pyplot.show()
    pyplot.close()

In [ ]:
summarize_diagnostics(history)

NameError: ignored

In [ ]:
#predicting the next word using an initial sentence
input_phrase = "And they"
next_words = 10
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([input_phrase])[0] #converting our input_phrase to tokens and excluding the out of vcabulary words
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') #padding the input_phrase
	predicted = model.predict_classes(token_list, verbose=0) #predicting the token of the next word using our trained model
	output_word = "" #initialising output word as blank at the beginning
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word #converting the token back to the corresponding word and storing it in the output_word
			break
	input_phrase += " " + output_word
print(input_phrase)

To start and the elves tell had the hobbits were before hatred
